In [1]:
import sys
from pathlib import Path

import pandas as pd

print(sys.executable)
print(pd.__version__)

c:\1_Projekte\quantum_cognition\.venv\Scripts\python.exe
2.3.3


In [6]:

# =============================================================================
# 1. Pfade anpassen: Basis-VTT-Ordner & Zielordner für CSV
# =============================================================================
INPUT_ROOT = Path(r"C:\1_Projekte\quantum_cognition\data\1_transcripts_from_mp3")
OUTPUT_DIR = Path(r"C:\1_Projekte\quantum_cognition\data\1-2_trancsript_panel")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV = OUTPUT_DIR / "transcript_panel.csv"


In [7]:
# =============================================================================
# 2. Hilfsfunktion: Ein VTT-File in Cue-Records (Panel-Zeilen) umwandeln
# =============================================================================
def parse_vtt_file(vtt_path: Path) -> list[dict]:
    """Parst eine VTT-Datei (Whisper-Output o.Ä.) in eine Liste von Segment-Dictionaries.

    Jede Zeile entspricht einem Cue (Segment) mit Startzeit, Endzeit und Text.
    """
    records = []
    with vtt_path.open("r", encoding="utf-8", errors="replace") as f:
        lines = [line.rstrip("\n") for line in f]

    i = 0
    segment_index = 0

    while i < len(lines):
        line = lines[i].strip()

        # Header und leere Zeilen überspringen
        if not line or line.upper().startswith("WEBVTT"):
            i += 1
            continue

        # Optional: numerische Cue-ID (1, 2, 3, ...)
        if line.isdigit():
            i += 1
            continue

        # Zeitstempel-Zeile erkennen
        if "-->" in line:
            time_part = line.split("-->")
            start_raw = time_part[0].strip()
            end_raw = time_part[1].strip().split(" ")[0]  # Einstellungen nach Leerzeichen ignorieren

            i += 1
            text_lines = []

            # Alle folgenden Textzeilen bis zur nächsten Leerzeile einsammeln
            while i < len(lines) and lines[i].strip():
                text_lines.append(lines[i].strip())
                i += 1

            text = " ".join(text_lines)

            records.append(
                {
                    "folder_relative": str(vtt_path.parent.relative_to(INPUT_ROOT)),
                    "file_name": vtt_path.name,
                    "segment_index": segment_index,
                    "start_time": start_raw,
                    "end_time": end_raw,
                    "text": text,
                }
            )
            segment_index += 1
        else:
            # Sonstige Zeilen (Sonderfälle) überspringen
            i += 1

    return records


In [8]:
# =============================================================================
# 3. Alle VTT-Dateien rekursiv einlesen und zu einem Panel-DataFrame vereinen
# =============================================================================
all_records: list[dict] = []

for vtt_file in INPUT_ROOT.rglob("*.vtt"):
    # Optional: nur genau eine Unterordner-Schicht erwartet, rglob ist aber robuster
    file_records = parse_vtt_file(vtt_file)
    all_records.extend(file_records)

# Panel aufbauen
df_panel = pd.DataFrame(all_records)

# Konsistente Sortierung (z.B. nach Ordner, Datei, Segment)
df_panel.sort_values(["folder_relative", "file_name", "segment_index"], inplace=True)
df_panel.reset_index(drop=True, inplace=True)

# =============================================================================
# 4. Panel als CSV exportieren
# =============================================================================
df_panel.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")

print(f"Fertig. Panel mit {len(df_panel)} Zeilen exportiert nach:\n{OUTPUT_CSV}")

Fertig. Panel mit 28157 Zeilen exportiert nach:
C:\1_Projekte\quantum_cognition\data\1-2_trancsript_panel\transcript_panel.csv
